In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
import time
import concurrent.futures
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import mplfinance as mpf
import plotly.graph_objects as go
import nbformat
import plotly.io as pio
from sklearn.preprocessing import StandardScaler
from TradingModelTrainer import TradingModelTrainer
from DataProvider import DataProvider
from GeneticAlgorithm import GeneticAlgorithm



In [2]:
def save_models(models: list[Sequential], folder: str = "Models"):
    """Save models to a specified folder."""
    if not os.path.exists(folder):
        os.makedirs(folder)
    for i, model in enumerate(models):
        model_path = os.path.join(folder, f"model_{i + 1}.h5")
        model.save(model_path)
        print(f"Model {i + 1} saved at {model_path}")

def load_models(model_names: list[str], folder: str = "Models") -> list[Sequential]:
    """Load models from a specified folder."""
    loaded_models = []
    for model_name in model_names:
        model_path = os.path.join(folder, model_name)
        if os.path.exists(model_path):
            model = load_model(model_path)
            loaded_models.append(model)
            print(f"Loaded model from {model_path}")
        else:
            print(f"Model file {model_path} not found.")
    return loaded_models

In [3]:
data_provider = DataProvider('Data/Binance_BTCUSDT_2024_minute.csv')
raw_df = data_provider.get_raw_data()[:40000]
normalized_df = data_provider.get_normalized_data()[:40000]


In [4]:
TIMESTEPS = 100  # Adjust based on your data's characteristics
FEATURES = 7    # As determined from your data
POPULATION_SIZE = 100  # Number of models in each generation
GENERATIONS = 20  # Number of generations for evolving models

# Initialize the TradingModelTrainer
# trainer = TradingModelTrainer(trainData=normalized_df,  # Replace with your training data DataFrame
#                               tradeData=raw_df,  # Replace with your trading data DataFrame
#                               initial_capital=10000,   # Example initial capital
#                               trade_size_dollars=5000) # Example trade size

# ga = GeneticAlgorithm(trainer)

In [5]:
# population = load_models(["model_1.h5", "model_2.h5", "model_3.h5", "model_4.h5"])

In [6]:
trainer = TradingModelTrainer(trainData=normalized_df,
                            tradeData=raw_df,
                            initial_capital=10000,  
                            trade_size_dollars=5000) 

ga = GeneticAlgorithm(trainer)
while ga.best_sharpe_ratio < 0:
    run =1
    population = []
    if run == 1:        
        # population = load_models(["model_1.h5"], "1")
        for _ in range(POPULATION_SIZE):
            model = trainer.create_model(TIMESTEPS, FEATURES)
            population.append(model)
    else:
        # For subsequent runs, load models from the previous run
        population = load_models(["model_1.h5", "model_2.h5", "model_3.h5", "model_4.h5", "model_5.h5"], str(run - 1))
        for _ in range(POPULATION_SIZE):
            model = trainer.create_model(TIMESTEPS, FEATURES)
            population.append(model)

    final_population = ga.genetic_algorithm(population, GENERATIONS, TIMESTEPS, FEATURES)
    save_models(final_population[:5], str(run))
    run += 1
#prev bst
    


gen: 0 / 20
Generation 1 complete. Best Sharpe Ratio: -4.4945644630314225
gen: 1 / 20
Generation 2 complete. Best Sharpe Ratio: -6.784474222005307
gen: 2 / 20


ResourceExhaustedError: Graph execution error:

SameWorkerRecvDone unable to allocate output tensor. Key: /job:localhost/replica:0/task:0/device:CPU:0;d9e12db2d81ba370;/job:localhost/replica:0/task:0/device:GPU:0;edge_68_IteratorGetNext;0:0
	 [[{{node IteratorGetNext/_4}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_predict_function_1072843]

: 

In [ ]:
learning_df = trainer.get_learning_history_df()
learning_df

In [ ]:
trainer = TradingModelTrainer(trainData=normalized_df,
                            tradeData=raw_df,
                            initial_capital=10000,  
                            trade_size_dollars=5000) 

ga = GeneticAlgorithm(trainer)
traders = []

for model in load_models(["model_1.h5", "model_2.h5", "model_3.h5", "model_4.h5"], "5"):
    trader = trainer.simulate_trading(model, TIMESTEPS)
    traders.append(trader)

In [ ]:
trader = traders[2]
result = trader.get_trade_history_df()
# final_population now contains the evolved models
result

In [ ]:
result.describe()

In [ ]:

trader.calculate_sharpe_ratio()

In [ ]:
trader.current_capital

In [ ]:
# # Sample data for demonstration
# import pandas as pd

# from Trader import Trader


# data = {
#     'close': [100, 105, 110, 105, 100, 95, 90, 95, 100, 80]  # Simulated close prices
# }
# df = pd.DataFrame(data)

# # Initialize the Trader
# trade_size_dollars = 1000  # Trade size in dollars
# initial_capital = 10000    # Initial capital in dollars
# trader = Trader(df, trade_size_dollars, initial_capital)

# # Define a series of actions: 0 for long, 1 for wait, 2 for short
# actions = [0, 1, 2, 1, 1, 1, 0, 1, 1, 2]  # Example sequence of actions

# # Simulate trading
# trader.trade(actions)

# # Get the trade history
# trade_history_df = trader.get_trade_history_df()
# trade_history_df